In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional, Flatten, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from scipy.io import loadmat

In [ ]:
SNOMED_scored=pd.read_csv("/kaggle/input/physionet-snomed-mappings/SNOMED_mappings_scored.csv", sep=";")
SNOMED_unscored=pd.read_csv("/kaggle/input/physionet-snomed-mappings/SNOMED_mappings_unscored.csv", sep=";")

In [ ]:
snomed_class_names=["Pacing Rhythm", "Prolonged QT Interval","Atrial Fibrillation","Atrial Flutter",
                 "Left Bundle Branch Block","Q Wave Abnormal","T Wave Abnormal","Prolonged PR Interval","Ventricular Premature Beats",
"Low QRS Voltages","1st Degree AV Block","Premature Atrial Contraction","Left Axis Deviation",
"Sinus Bradycardia","Bradycardia","Sinus Rhythm","Sinus Tachycardia","Premature Ventricular Contractions",
"Sinus Arrhythmia","Left Anterior Fascicular Block","Right Axis Deviation","Right Bundle Branch Block","T Wave Inversion",
"Supraventricular Premature Beats","Nonspecific Intraventricular Conduction Disorder","Incomplete Right Bundle Branch Block",
"Complete Right Bundle Branch Block"]

In [ ]:
header_file="../input/ptbxl-electrocardiography-database/WFDB/HR00002.hea"
mat_file="../input/ptbxl-electrocardiography-database/WFDB/HR00002.mat"

In [ ]:
import pandas as pd
with open(header_file, 'r') as the_file:
    all_data = [line.strip() for line in the_file.readlines()]
    data = all_data[8:]
snomed_number=int(data[7][5:14])
value_unscored=SNOMED_unscored["Dx"][SNOMED_unscored["SNOMED CT Code"]==snomed_number].values
value_scored=SNOMED_scored["Dx"][SNOMED_scored["SNOMED CT Code"]==snomed_number].values
try: 
    disease_unscored=value_unscored[0]
except:
    disease_unscored=""

try: 
    disease_scored=value_scored[0]
except:
    disease_scored=""

### Results

# Simple ANN

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(50, activation='relu', input_shape=(5000,12)))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(GlobalAveragePooling1D())
ann_model.add(Dense(27, activation='softmax'))

In [ ]:
ann_model.load_weights("../input/ecg-models/ann_model_weights.best.hdf5")

In [ ]:
yhat=ann_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# CNN

# Lenet-5

In [ ]:
lenet_5_model=Sequential()

lenet_5_model.add(Conv1D(filters=64, kernel_size=5, padding='same', input_shape=(5000,12)))
#lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))

lenet_5_model.add(Conv1D(filters=64, kernel_size=3, padding='same',))
#lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))

lenet_5_model.add(GlobalAveragePooling1D())

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(27, activation = 'softmax'))

In [ ]:
lenet_5_model.load_weights("../input/ecg-models/lenet5_model_weights.best.hdf5")

In [ ]:
yhat=lenet_5_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# AlexNet

In [ ]:
alexNet_model=Sequential()

alexNet_model.add(Conv1D(filters=96, kernel_size=11, padding='same', input_shape=(5000,12)))
#alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))

alexNet_model.add(Conv1D(filters=256, kernel_size=5, padding='same'))
#alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))

alexNet_model.add(Conv1D(filters=384, padding='same', activation='relu', kernel_size=3))
alexNet_model.add(Conv1D(filters=384, activation='relu', kernel_size=3))
alexNet_model.add(Conv1D(filters=256, kernel_size=3))
#alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))

alexNet_model.add(GlobalAveragePooling1D())
alexNet_model.add(Dense(64, activation='relu'))
alexNet_model.add(Dropout(0.25))
alexNet_model.add(Dense(64, activation='relu'))
alexNet_model.add(Dropout(0.25))
alexNet_model.add(Dense(27, activation='softmax'))

In [ ]:
alexNet_model.load_weights("../input/ecg-models/alexnet_model_weights.best.hdf5")

In [ ]:
yhat=alexNet_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# VGG-16

In [ ]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(5000,12)))
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(GlobalAveragePooling1D())
vgg_16_model.add(Dense(256, activation='relu'))
vgg_16_model.add(Dropout(0.25))
vgg_16_model.add(Dense(128, activation='relu'))
vgg_16_model.add(Dropout(0.25))
vgg_16_model.add(Dense(27, activation='softmax'))

In [ ]:
vgg_16_model.load_weights("../input/ecg-models/vgg16_model_weights.best.hdf5")

In [ ]:
yhat=vgg_16_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# ResNet50

In [ ]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv1D(filters = F1, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(filters = F2, kernel_size = f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(filters = F3, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)

    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, s = 2):
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv1D(F1, 1, activation='relu', strides = s)(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(F2, f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(F3, 1, strides = 1, activation='relu')(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv1D(F3, 1, strides = s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def ResNet50(input_shape):
    
    X_input = Input(input_shape)

    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(64, 7, strides = 2, activation='relu')(X)
    X = BatchNormalization()(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(27,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
resNet50_model = ResNet50(input_shape = (5000,12))

In [ ]:
resNet50_model.load_weights("../input/ecg-models/resnet50_model_weights.best.hdf5")

In [ ]:
yhat=resNet50_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# Inception

In [ ]:
def inception_block(prev_layer):
    
    conv1=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv1=BatchNormalization()(conv1)
    conv1=Activation('relu')(conv1)
    
    
    conv3=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    
    conv3=Conv1D(filters = 64, kernel_size = 3, padding = 'same')(conv3)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
   
    
    conv5=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    
    conv5=Conv1D(filters = 64, kernel_size = 5, padding = 'same')(conv5)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    
    pool= MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(pool)
    convmax=BatchNormalization()(convmax)
    convmax=Activation('relu')(convmax)
    
    layer_out = concatenate([conv1, conv3, conv5, convmax], axis=1)
    
    return layer_out

def inception_model(input_shape):
    X_input=Input(input_shape)
    
    #X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X_input)
    #X = MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    
    
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    """X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)
    X = inception_block(X)
    X = MaxPool1D(pool_size=2, strides=4, padding='same')(X)"""
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(64,activation='relu')(X)
    X = Dense(64,activation='relu')(X)
    X = Dense(27,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [ ]:
inception_model = inception_model(input_shape = (5000,12))

In [ ]:
inception_model.load_weights("../input/ecg-models/inception_model2_weights.best.hdf5")

In [ ]:
yhat=inception_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

# RNN

# LSTM

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, return_sequences=True, input_shape=(5000,12)))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32, return_sequences=True))
lstm_model.add(GlobalAveragePooling1D())
lstm_model.add(Dense(32, activation = 'relu'))
lstm_model.add(Dense(27, activation = 'softmax'))

In [ ]:
lstm_model.load_weights("../input/ecg-models/lstm_model_weights.best.hdf5")

In [ ]:
yhat=lstm_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)